<a href="https://colab.research.google.com/github/ayulockin/debugNNwithWandB/blob/master/Cifar10_pytorch_wandb_Wt_Init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and Setups

In [1]:
!pip install wandb -q

     |████████████████████████████████| 1.4MB 16.6MB/s 
     |████████████████████████████████| 460kB 45.2MB/s 
     |████████████████████████████████| 92kB 10.7MB/s 
     |████████████████████████████████| 102kB 12.9MB/s 
     |████████████████████████████████| 102kB 12.6MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 


In [0]:
import wandb

In [3]:
!wandb login

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 69f60a7711ce6b8bbae91ac6d15e45d6b1f1430e
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

#### For GPU

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## CIFAR10 Dataset

In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

0it [00:00, ?it/s]

 99%|█████████▉| 168402944/170498071 [00:14<00:00, 14038255.19it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [28]:
trainset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [7]:
iter(trainloader).next()[0][0].shape

torch.Size([3, 32, 32])

## Model



In [8]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf

In [10]:
inputs = tf.keras.layers.Input(shape=(32,32,3))
x = tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='valid')(inputs)
x = tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='valid')(inputs)
x = tf.keras.layers.MaxPool2D((2,2))(x)
x = tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = tf.keras.layers.MaxPool2D((2,2))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.models.Model(inputs, x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0     

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 32, 3, 1)
        self.conv3 = nn.Conv2d(32, 64, 3, 1)
        self.conv4 = nn.Conv2d(64, 64, 3, 1)

        self.pool1 = torch.nn.MaxPool2d(2)
        self.pool2 = torch.nn.MaxPool2d(2)

        self.fc1 = nn.Linear(1600, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        ## Conv 1st Block
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
def train(model, device, train_loader, optimizer, epoch, steps_per_epoch):
  # Switch model to training mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.train()
  train_total = 0
  train_correct = 0

  # We loop over the data iterator, and feed the inputs to the network and adjust the weights.
  for batch_idx, (data, target) in enumerate(train_loader, start=0):
    if batch_idx > steps_per_epoch:
      break
    # Load the input features and labels from the training dataset
    data, target = data.to(device), target.to(device)
    
    # Reset the gradients to 0 for all learnable weight parameters
    optimizer.zero_grad()
    
    # Forward pass: Pass image data from training dataset, make predictions about class image belongs to (0-9 in this case)
    output = model(data)
    
    # Define our loss function, and compute the loss
    loss = F.nll_loss(output, target)

    scores, predictions = torch.max(output.data, 1)
    train_total += target.size(0)
    train_correct += int(sum(predictions == target))
            
    # Backward pass: compute the gradients of the loss w.r.t. the model's parameters
    loss.backward()
    
    # Update the neural network weights
    optimizer.step()

  acc = round((train_correct / train_total) * 100, 2)
  print('Epoch [{}], Loss: {}, Accuracy: {}, '.format(epoch, loss.item(), acc), end='')
  wandb.log({'Train Loss': loss.item(), 'Train Accuracy': acc})


In [0]:
def test(model, device, test_loader, classes):
  # Switch model to evaluation mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.eval()
  
  test_loss = 0
  test_total = 0
  test_correct = 0

  with torch.no_grad():
      for data, target in test_loader:
          # Load the input features and labels from the test dataset
          data, target = data.to(device), target.to(device)
          
          # Make predictions: Pass image data from test dataset, make predictions about class image belongs to (0-9 in this case)
          output = model(data)
          
          # Compute the loss sum up batch loss
          test_loss += F.nll_loss(output, target, reduction='sum').item()
          
          scores, predictions = torch.max(output.data, 1)
          test_total += target.size(0)
          test_correct += int(sum(predictions == target))
          
  acc = round((test_correct / test_total) * 100, 2)
  print(' Test_loss: {}, Test_accuracy: {}'.format(test_loss/test_total, acc))
  wandb.log({'Test Loss': test_loss/test_total, 'Test Accuracy': acc})


## Let's train it

In [45]:
net = Net().to(device)
print(net)

optimizer = optim.Adam(net.parameters())

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [46]:
wandb.init(project='wtinit')
wandb.watch(net, log='all')

for epoch in range(10):
  train(net, device, trainloader, optimizer, epoch, steps_per_epoch=50000//32)
  test(net, device, testloader, classes)

print('Finished Training')

Epoch [0], Loss: 0.7660771608352661, Accuracy: 45.75,  Test_loss: 1.2285920398712158, Test_accuracy: 55.95
Epoch [1], Loss: 0.7466053366661072, Accuracy: 62.81,  Test_loss: 0.9519663627624512, Test_accuracy: 66.44
Epoch [2], Loss: 1.199931263923645, Accuracy: 70.17,  Test_loss: 0.8615626026153564, Test_accuracy: 70.16
Epoch [3], Loss: 1.124739170074463, Accuracy: 75.22,  Test_loss: 0.8328243703842163, Test_accuracy: 70.87
Epoch [4], Loss: 0.7760188579559326, Accuracy: 78.84,  Test_loss: 0.8024488975524903, Test_accuracy: 73.56
Epoch [5], Loss: 0.7488182783126831, Accuracy: 82.23,  Test_loss: 0.8036666326522827, Test_accuracy: 74.02
Epoch [6], Loss: 0.5187238454818726, Accuracy: 85.23,  Test_loss: 0.8454585586547851, Test_accuracy: 73.1
Epoch [7], Loss: 0.2754251956939697, Accuracy: 87.31,  Test_loss: 0.8719929852485657, Test_accuracy: 73.46
Epoch [8], Loss: 0.2809068560600281, Accuracy: 89.74,  Test_loss: 1.0231557291984559, Test_accuracy: 72.31
Epoch [9], Loss: 0.1009332686662674, Acc

## He Init


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        torch.nn.init.kaiming_uniform_(self.conv1.weight, mode='fan_in', nonlinearity='relu')
        self.conv2 = nn.Conv2d(32, 32, 3, 1)
        torch.nn.init.kaiming_uniform_(self.conv2.weight, mode='fan_in', nonlinearity='relu')
        self.conv3 = nn.Conv2d(32, 64, 3, 1)
        torch.nn.init.kaiming_uniform_(self.conv3.weight, mode='fan_in', nonlinearity='relu')
        self.conv4 = nn.Conv2d(64, 64, 3, 1)
        torch.nn.init.kaiming_uniform_(self.conv4.weight, mode='fan_in', nonlinearity='relu')

        self.pool1 = torch.nn.MaxPool2d(2)
        self.pool2 = torch.nn.MaxPool2d(2)

        self.fc1 = nn.Linear(1600, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        ## Conv 1st Block
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
del net

In [52]:
net = Net().to(device)
optimizer = optim.Adam(net.parameters())

wandb.init(project='wtinit')
wandb.watch(net, log='all')

for epoch in range(10):
  train(net, device, trainloader, optimizer, epoch, steps_per_epoch=50000//32)
  test(net, device, testloader, classes)

print('Finished Training')

Epoch [0], Loss: 0.8680363893508911, Accuracy: 53.15,  Test_loss: 0.9655896467208862, Test_accuracy: 65.97
Epoch [1], Loss: 0.9842775464057922, Accuracy: 69.86,  Test_loss: 0.8211903447151184, Test_accuracy: 71.67
Epoch [2], Loss: 0.7356349229812622, Accuracy: 75.92,  Test_loss: 0.7943427026748657, Test_accuracy: 73.04
Epoch [3], Loss: 0.47701454162597656, Accuracy: 80.88,  Test_loss: 0.7719963906049728, Test_accuracy: 74.26
Epoch [4], Loss: 0.5376359224319458, Accuracy: 85.06,  Test_loss: 0.8065440139770508, Test_accuracy: 74.54
Epoch [5], Loss: 0.16438741981983185, Accuracy: 88.42,  Test_loss: 0.8578581342697144, Test_accuracy: 75.66
Epoch [6], Loss: 0.1592894345521927, Accuracy: 91.03,  Test_loss: 0.9687001671791077, Test_accuracy: 74.76
Epoch [7], Loss: 0.11241734027862549, Accuracy: 93.03,  Test_loss: 1.1693744165897368, Test_accuracy: 72.9
Epoch [8], Loss: 0.027939707040786743, Accuracy: 94.07,  Test_loss: 1.2283352478027343, Test_accuracy: 72.83
Epoch [9], Loss: 0.25541281700134